### imports and setup

In [1]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

In [2]:
from utils.jraph_data import get_lorenz_graph_tuples, print_graph_fts
from utils.jraph_training import rollout_loss, train_step, train_step_fn, evaluate_model, train_and_evaluate, create_dataset #, rollout_loss_batched, 
from utils.jraph_models import MLPBlock, MLPGraphNetwork
import optax
from flax.training import train_state
from flax_gnn_example.configs import mlpblock_test

import numpy as np
import jax.numpy as jnp
import jax
from datetime import datetime

2023-11-17 00:27:51.756959: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# set up logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### test the gn core model

In [4]:
from tests.helpers import get_sample_data, state_setup_helper
def forward_pass_helper_test(model):
    """ Helper function to test the forward pass for an arbitrary model. 
    
    """
    # get sample graph
    sample_dataset, _ = get_sample_data()
    sample_input_window = sample_dataset['train']['inputs'][0]
    sample_target_window = sample_dataset['train']['targets'][0]

    # set up state object, which helps us keep track of the model, params, and optimizer
    state = state_setup_helper(model)

    pred_graph = state.apply_fn(state.params, sample_input_window)

    return state, pred_graph, sample_dataset
    # first_target_graph = sample_target_window[0]

    # # check that the forward_pass_helper_test shape of the node features is correct
    # self.assertEqual(pred_graph.nodes.shape, first_target_graph.nodes.shape)

    # # check edge features did not change
    # self.assertTrue(
    #     jnp.array_equal(pred_graph.edges, first_target_graph.edges))

    # # check global features did not change
    # self.assertTrue(
    #     jnp.array_equal(pred_graph.globals, first_target_graph.globals))



In [7]:
model1 = MLPGraphNetwork(n_blocks=1, share_params=False)

# get sample graph
sample_dataset, _ = get_sample_data()
sample_input_window = sample_dataset['train']['inputs'][0]
sample_target_window = sample_dataset['train']['targets'][0]

In [8]:
# set up state object, which helps us keep track of the model, params, and optimizer
rng = jax.random.key(0)
rng, init_rng = jax.random.split(rng)
params1 = jax.jit(model1.init)(init_rng, sample_input_window)

> /Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/utils/jraph_models.py(174)__call__()
    172         # Apply a Graph Network once for each message-passing round.
    173         pdb.set_trace()
--> 174         processed_graphs = nn.Sequential(blocks)(input_window_graphs)
    175         # TODO: do we need skip connections or layer_norm here?
    176 

[GraphsTuple(nodes=Traced<ShapedArray(float32[36,2])>with<DynamicJaxprTrace(level=1/0)>, edges=Traced<ShapedArray(float32[180,1])>with<DynamicJaxprTrace(level=1/0)>, receivers=Traced<ShapedArray(int32[180])>with<DynamicJaxprTrace(level=1/0)>, senders=Traced<ShapedArray(int32[180])>with<DynamicJaxprTrace(level=1/0)>, globals=Traced<ShapedArray(float32[1,1])>with<DynamicJaxprTrace(level=1/0)>, n_node=Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace(level=1/0)>, n_edge=Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace(level=1/0)>), GraphsTuple(nodes=Traced<ShapedArray(float32[36,2])>with<DynamicJaxprTrace(level=1/0)>, edge

In [10]:
model2 = MLPGraphNetwork(n_blocks=2, share_params=False)

params2 = jax.jit(model2.init)(init_rng, sample_input_window)

> /Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/utils/jraph_models.py(174)__call__()
    172         # TODO: do we need skip connections or layer_norm here?
    173 
--> 174         return processed_graphs_list
    175 
    176 



In [11]:
model10 = MLPGraphNetwork(n_blocks=10, share_params=False)
params10 = jax.jit(model10.init)(init_rng, sample_input_window)

> /Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/utils/jraph_models.py(174)__call__()
    172         # TODO: do we need skip connections or layer_norm here?
    173 
--> 174         return processed_graphs_list
    175 
    176 



In [14]:
params10['params'].keys()

dict_keys(['MLPBlock_0', 'MLPBlock_1', 'MLPBlock_2', 'MLPBlock_3', 'MLPBlock_4', 'MLPBlock_5', 'MLPBlock_6', 'MLPBlock_7', 'MLPBlock_8', 'MLPBlock_9'])